<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Deep-Learning/blob/main/CNN-Sentiment-Analysis-Moview-Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>